In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
# os.chdir('/content/drive/MyDrive/cheak_detaction/yolov8')
os.chdir('/content/drive/MyDrive/customer_service')

In [ ]:
!pip install ultralytics

In [ ]:
import glob
import numpy as np
from PIL import Image
import os
# from sklearn.model_selection import train_test_split
import cv2
import os
from ultralytics import YOLO
import shutil
# from skimage.io import imread_collection
import torch
# import tensorflow as tf
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib

In [ ]:
# tf.debugging.set_log_device_placement(True)


# device: str = "mps" if torch.backends.mps.is_available() else "cpu"


model_11 = YOLO("Copy of first_crop.pt")
# model_1.to('cuda')
# model_11.to(device)
model_2 = YOLO("Copy of Copy of best.pt")
# model_2.to(device)

# characterRecognition = tf.keras.models.load_model("Copy of model_char_recognition_4.h5")
my_model = YOLO('best_seg.pt')
# my_model.to(device)

In [ ]:
class TinyVGG(nn.Module):
    """
    Model architecture copying TinyVGG from:
    https://poloclub.github.io/cnn-explainer/
    """
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
        super().__init__()
        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape,
                      out_channels=hidden_units,
                      kernel_size=3, # how big is the square that's going over the image?
                      stride=1, # default
                      padding=1), # options = "valid" (no padding) or "same" (output has same shape as input) or int for specific number
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,
                         stride=2) # default stride value is same as kernel_size
        )
        self.conv_block_2 = nn.Sequential(
            nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            # Where did this in_features shape come from?
            # It's because each layer of our network compresses and changes the shape of our inputs data.
            nn.Linear(in_features=hidden_units*16*16,
                      out_features=output_shape)
        )

    def forward(self, x: torch.Tensor):
        x = self.conv_block_1(x)
        # print(x.shape)
        x = self.conv_block_2(x)
        # print(x.shape)
        x = self.classifier(x)
        # print(x.shape)
        return x
        # return self.classifier(self.conv_block_2(self.conv_block_1(x))) # <- leverage the benefits of operator fusion

# Create model_1 and send it to the target device
torch.manual_seed(42)
model_1 = TinyVGG(
    input_shape=3,
    hidden_units=10,
    output_shape=10)

model_1.load_state_dict(torch.load(r'01_pytorch_workflow_model_1.pth',map_location=torch.device('cpu') ))


# def cnnCharRecognition(img):
#     dictionary = {0:'0', 1:'1', 2 :'2', 3:'3', 4:'4', 5:'5', 6:'6', 7:'7', 8:'8', 9:'9', 10:'A',
#     11:'B', 12:'C', 13:'D', 14:'E', 15:'F', 16:'G', 17:'H', 18:'I', 19:'J', 20:'K',
#     21:'L', 22:'M', 23:'N', 24:'P', 25:'Q', 26:'R', 27:'S', 28:'T', 29:'U',
#     30:'V', 31:'W', 32:'X', 33:'Y', 34:'Z'}

#     blackAndWhiteChar=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     blackAndWhiteChar = cv2.resize(blackAndWhiteChar,(75,100))
#     image = blackAndWhiteChar.reshape((1,100,75,1))
#     image = image / 255.0
#     new_predictions = characterRecognition.predict(image)
#     char = np.argmax(new_predictions)
#     return char

def number_recog(custom_image_path):
  # Load in custom image and convert the tensor values to float32
  custom_image = torchvision.io.read_image(str(custom_image_path)).type(torch.float32)

  # Divide the image pixel values by 255 to get them between [0, 1]
  custom_image = custom_image / 255.
  # Create transform pipleine to resize image
  custom_image_transform = transforms.Compose([
      transforms.Resize((64, 64)),
  ])

  # Transform target image
  custom_image_transformed = custom_image_transform(custom_image)
  model_1.eval()
  with torch.inference_mode():
      # Add an extra dimension to image
      custom_image_transformed_with_batch_size = custom_image_transformed.unsqueeze(dim=0)

      # Print out different shapes
      print(f"Custom image transformed shape: {custom_image_transformed.shape}")
      print(f"Unsqueezed custom image shape: {custom_image_transformed_with_batch_size.shape}")

      # Make a prediction on image with an extra dimension
      custom_image_pred = model_1(custom_image_transformed.unsqueeze(dim=0))
  # Convert logits -> prediction probabilities (using torch.softmax() for multi-class classification)
  custom_image_pred_probs = torch.softmax(custom_image_pred, dim=1)
  # Convert prediction probabilities -> prediction labels
  custom_image_pred_label = torch.argmax(custom_image_pred_probs, dim=1)
  class_names = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
  # Find the predicted label
  custom_image_pred_class = class_names[custom_image_pred_label.cpu()] # put pred label to CPU, otherwise will error
  return custom_image_pred_class


def first_crop(img):
      # try:
    os.chdir('/content/drive/MyDrive/customer_service')
    # detect location of plate
    img1 = 'images/'+img
    result = model_11.predict(source=img1,save_crop = True)
    # change directory to detected and cropd plate
    os.chdir('runs/detect/predict/crops/0')
     # detect characters and numbers
    img = img[:-3]+'jpg'
    result = model_2.predict(source=img , save_crop = True)
    result_char = my_model.predict(source=img )
    # change directory to detected and croped characters and numbers
    os.chdir('runs/detect/predict/crops/number')

    #access location of detected characters
    for r in result:
      crd = r.boxes.xyxy
    coordinates = crd.tolist()
    print(coordinates)

    lst = [img]
    dic = {img:coordinates[0][0]}
    for i in range(2,9):
      cropped_name = img[:-4]+ str(i) +'.jpg'
      dic[cropped_name] = coordinates[i-1][0]

    print(dic)
    #sort croppd characters and numbers by their locations
    dic = {k: v for k, v in sorted(dic.items(), key=lambda item: item[1])}

    #character recognetion and classification
    lst_rs=[]
    for i in dic:
      custom_image_path = i
      lst_rs.append(number_recog(custom_image_path))
      # print(cnnCharRecognition(im))
      print(i)
      print("-----------------")
    names = {0: 'a', 1: 'b', 2: 'd', 3: 'eight', 4: 'ein', 5: 'five', 6: 'four',
          7: 'ghaf', 8: 'h', 9: 'jim', 10: 'lam', 11: 'mim', 12: 'nine',
          13: 'non', 14: 'one', 15: 'sad', 16: 'seven', 17: 'sin', 18: 'six', 19: 'ta',
          20: 'three', 21: 'two', 22: 'waw', 23: 'wheel', 24: 'y', 25: 'zero'}
    for r in result_char:
      cls_names = r.boxes.cls.tolist()

    lst = [0,1,2,4,7,8,9,10,11,12,13,14,15,17,19,22,23,24]
    for i in lst :
      if i in cls_names :
        lst_rs[2] = names[i]
    # lst_rs[2] =  charac
    # Deleting an non-empty folder
    dir_path = r"/content/drive/MyDrive/customer_service/runs"
    shutil.rmtree(dir_path, ignore_errors=True)
    os.chdir('/content/drive/MyDrive/customer_service')
    return(str(lst_rs))

In [ ]:
#dont run
first_crop('65ya61677.png')


image 1/1 /content/drive/MyDrive/customer_service/images/65ya61677.png: 480x640 1 0, 1497.5ms
Speed: 9.8ms preprocess, 1497.5ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
Results saved to runs/detect/predict

image 1/1 /content/drive/MyDrive/customer_service/runs/detect/predict/crops/0/65ya61677.jpg: 288x640 8 numbers, 632.1ms
Speed: 1.9ms preprocess, 632.1ms inference, 0.9ms postprocess per image at shape (1, 3, 288, 640)
Results saved to runs/detect/predict

image 1/1 /content/drive/MyDrive/customer_service/runs/detect/predict/crops/0/65ya61677.jpg: 288x640 1 y, 2455.9ms
Speed: 1.5ms preprocess, 2455.9ms inference, 3.3ms postprocess per image at shape (1, 3, 288, 640)
[[98.92752075195312, 21.81349754333496, 109.53571319580078, 46.82591247558594], [110.79866027832031, 22.025680541992188, 121.64578247070312, 46.629478454589844], [20.992311477661133, 14.450210571289062, 29.895048141479492, 41.38693618774414], [61.67079162597656, 16.0031795501709, 72.43990325927734

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


"['6', '5', 'y', '6', '1', '6', '7', '7']"

In [ ]:
import os
# os.chdir('/content/drive/MyDrive/cheak_detaction/yolov8')
os.chdir('/content/drive/MyDrive/customer_service')

In [ ]:
!pip install fastapi uvicorn pyngrok


In [ ]:
!pip install python-multipart

In [ ]:
from fastapi import FastAPI, Request, UploadFile, File
from fastapi.responses import HTMLResponse
from fastapi.templating import Jinja2Templates
from fastapi.staticfiles import StaticFiles
# from Anpr import read_imgs
# from Anpr import first_crop
# from test import first_crop
import cv2
# from cv2  import dnn_superres
import torch
# from skimage.io import imread_collection
from ultralytics import YOLO
# from cv2 import dnn_superres
import uvicorn
from typing import List
from pathlib import Path
from fastapi.responses import FileResponse
from ultralytics import YOLO


import os


# device: str = "mps" if torch.backends.mps.is_available() else "cpu"

# model_1 = YOLO("Copy of first_crop.pt")
# model_1.to(device)
# # model_2 = YOLO("Copy of Copy of best.pt")
# # model_2.to(device)


# characterRecognition = tf.keras.models.load_model("Copy of model_char_recognition_4.h5")
# my_model = YOLO('best_seg.pt')
# my_model.to(device)
# yolo = torch.hub.load('ultralytics/yolov5', 'custom', path='best.pt')

# my_model = YOLO('weights_segment/best.pt')
IMAGEDIR = "images/"

app = FastAPI()
# BASE_DIR = Path(__file__).resolve().parent

# templates = Jinja2Templates(directory=str(Path(BASE_DIR, 'templates')))
templates = Jinja2Templates(directory="templates")
app.mount("/images", StaticFiles(directory="images"), name="images")

@app.get('/', response_class=HTMLResponse)
def home(request: Request):
    return templates.TemplateResponse("index.html", {"request": request})


@app.post("/images")
def images(img1: bytes = File(...), img2: bytes = File(...)):
    # **do something**
    return {"message": "OK"}



@app.post("/upload-files")
async def create_upload_files(request: Request, files: List[UploadFile] = File(...)):
    for file in files:
        contents = await file.read()
        #save the file
        os.chdir('/content/drive/MyDrive/customer_service')
        name = file.filename
        with open(f"{IMAGEDIR}{file.filename}", "wb") as f:
            f.write(contents)

    # show = [file.filename for file in files]
    # resultt  = read_imgs(name)


    return show_result(Request,name)
    #return {"Result": "OK", "filenames": [file.filename for file in files]}
    # return resultt



@app.get("/show-files",response_class=HTMLResponse)
def show_result(request: Request,name):
    resultt=first_crop(name)
    result_dic={'numbers':resultt,'img_name':name}
    # return templates.TemplateResponse("show.html" ,{"request":Request,"res":result_dic})
    # return templates.TemplateResponse("show.html" ,{"request":Request,"resultt":resultt,"name":name})
    return resultt

# if __name__ == '__main__':
#     uvicorn.run(app, port=8000,host='127.0.0.1')

In [ ]:
!ngrok config add-authtoken 2ZniBIHz91cZTaPXjlSX1o9ydam_5Sfgm5Edg7ZtaN829Mn7E

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Public URL: https://a8cb-34-106-0-38.ngrok-free.app


INFO:     Started server process [19401]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     37.32.121.233:0 - "GET / HTTP/1.1" 200 OK
INFO:     37.32.121.233:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     37.32.121.233:0 - "POST /upload-files/ HTTP/1.1" 307 Temporary Redirect
INFO:     37.32.121.233:0 - "POST /upload-files/ HTTP/1.1" 307 Temporary Redirect

image 1/1 /content/drive/MyDrive/customer_service/images/411643_2.jpg: 480x640 1 0, 1104.3ms
Speed: 2.5ms preprocess, 1104.3ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)
Results saved to runs/detect/predict

image 1/1 /content/drive/MyDrive/customer_service/runs/detect/predict/crops/0/411643_2.jpg: 288x640 8 numbers, 1063.8ms
Speed: 1.8ms preprocess, 1063.8ms inference, 1.1ms postprocess per image at shape (1, 3, 288, 640)
Results saved to runs/detect/predict

image 1/1 /content/drive/MyDrive/customer_service/runs/detect/predict/crops/0/411643_2.jpg: 288x640 1 y, 2409.0ms
Speed: 1.7ms preprocess, 2409.0ms inference, 4.7ms postprocess per image at shape (1, 3, 288, 640)
[[121.2802658

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


INFO:     37.32.121.233:0 - "GET / HTTP/1.1" 200 OK
INFO:     37.32.121.233:0 - "POST /upload-files/ HTTP/1.1" 307 Temporary Redirect

image 1/1 /content/drive/MyDrive/customer_service/images/214351_2.jpg: 480x640 1 0, 1029.8ms
Speed: 2.0ms preprocess, 1029.8ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)
Results saved to runs/detect/predict

image 1/1 /content/drive/MyDrive/customer_service/runs/detect/predict/crops/0/214351_2.jpg: 288x640 8 numbers, 641.0ms
Speed: 1.9ms preprocess, 641.0ms inference, 0.9ms postprocess per image at shape (1, 3, 288, 640)
Results saved to runs/detect/predict

image 1/1 /content/drive/MyDrive/customer_service/runs/detect/predict/crops/0/214351_2.jpg: 288x640 1 y, 2352.2ms
Speed: 2.0ms preprocess, 2352.2ms inference, 2.9ms postprocess per image at shape (1, 3, 288, 640)
[[98.92752075195312, 21.81349754333496, 109.53571319580078, 46.82591247558594], [110.79866027832031, 22.025680541992188, 121.64578247070312, 46.629478454589844], [20.9

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [19401]


In [ ]:
pwd

UsageError: CWD no longer exists - please use %cd to change directory.


In [ ]:
img = '65ya61677.png'
img = img[:-3]+'jpg'
img

'65ya61677.jpg'

In [ ]:
# first_crop('65ya61677.png')
img  = '65ya61677.png'
# try:
os.chdir('/content/drive/MyDrive/customer_service')
# detect location of plate
img1 = 'images/'+img
result = model_11.predict(source=img1,save_crop = True)
# change directory to detected and cropd plate
os.chdir('runs/detect/predict3/crops/0')
  # detect characters and numbers
img = img[:-3]+'jpg'
result = model_2.predict(source=img , save_crop = True)
result_char = my_model.predict(source=img )
# change directory to detected and croped characters and numbers
os.chdir('runs/detect/predict/crops/number')

#access location of detected characters
for r in result:
  crd = r.boxes.xyxy
coordinates = crd.tolist()
print(coordinates)

lst = [img]
dic = {img:coordinates[0][0]}
for i in range(2,9):
  cropped_name = img[:-4]+ str(i) +'.jpg'
  dic[cropped_name] = coordinates[i-1][0]

print(dic)
#sort croppd characters and numbers by their locations
dic = {k: v for k, v in sorted(dic.items(), key=lambda item: item[1])}

#character recognetion and classification
lst_rs=[]
for i in dic:
  custom_image_path = i
  lst_rs.append(number_recog(custom_image_path))
  # print(cnnCharRecognition(im))
  print(i)
  print("-----------------")
names = {0: 'a', 1: 'b', 2: 'd', 3: 'eight', 4: 'ein', 5: 'five', 6: 'four',
      7: 'ghaf', 8: 'h', 9: 'jim', 10: 'lam', 11: 'mim', 12: 'nine',
      13: 'non', 14: 'one', 15: 'sad', 16: 'seven', 17: 'sin', 18: 'six', 19: 'ta',
      20: 'three', 21: 'two', 22: 'waw', 23: 'wheel', 24: 'y', 25: 'zero'}
for r in result_char:
  cls_names = r.boxes.cls.tolist()

lst = [0,1,2,4,7,8,9,10,11,12,13,14,15,17,19,22,23,24]
for i in lst :
  if i in cls_names :
    lst_rs[2] = names[i]
# lst_rs[2] =  charac
# Deleting an non-empty folder
dir_path = r"/content/drive/MyDrive/customer_service/runs"
shutil.rmtree(dir_path, ignore_errors=True)
str(lst_rs)


image 1/1 /content/drive/MyDrive/customer_service/images/65ya61677.png: 480x640 1 0, 1013.5ms
Speed: 2.8ms preprocess, 1013.5ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)
Results saved to runs/detect/predict3

image 1/1 /content/drive/MyDrive/customer_service/runs/detect/predict3/crops/0/65ya61677.jpg: 288x640 8 numbers, 616.1ms
Speed: 1.8ms preprocess, 616.1ms inference, 0.8ms postprocess per image at shape (1, 3, 288, 640)
Results saved to runs/detect/predict

image 1/1 /content/drive/MyDrive/customer_service/runs/detect/predict3/crops/0/65ya61677.jpg: 288x640 1 y, 3531.2ms
Speed: 1.8ms preprocess, 3531.2ms inference, 4.6ms postprocess per image at shape (1, 3, 288, 640)
[[98.92752075195312, 21.81349754333496, 109.53571319580078, 46.82591247558594], [110.79866027832031, 22.025680541992188, 121.64578247070312, 46.629478454589844], [20.992311477661133, 14.450210571289062, 29.895048141479492, 41.38693618774414], [61.67079162597656, 16.0031795501709, 72.43990325927

"['6', '5', 'y', '6', '1', '6', '7', '7']"